In [14]:
from bs4 import BeautifulSoup

import requests,re

import pandas as pd

from urllib.parse import urlparse

SyntaxError: invalid syntax (1824176088.py, line 7)

In [2]:
url_base='https://www.vghtc.gov.tw'

url_work_table='/Module/Recruitment?WebMenuID=9d005e46-411b-46fc-b438-e0bd561eba78'
url_admit = '/Module/Admission?WebMenuID=9d005e46-411b-46fc-b438-e0bd561eba78'

url=url_base+url_work_table

g=requests.get(url)

soup=BeautifulSoup(g.content, 'html.parser')

In [3]:
soup.find('table').find_all('tr')

[<tr>
 <th><div style="text-align:left;">更新日期</div></th>
 <th>徵才類別</th>
 <th><div style="text-align:left;">徵才項目</div></th>
 <th><div style="text-align:left;">報名截止日</div></th>
 </tr>,
 <tr>
 <td data-th="更新日期">2023/1/5</td>
 <td data-th="徵才類別">醫師</td>
 <td class="aboutNews" data-th="徵才項目"><a href="/Module/RecruitmentContent?RecruitmentID=6af62829-4ce3-4f1f-bc9a-4a9d8317df80&amp;WebMenuID=9d005e46-411b-46fc-b438-e0bd561eba78">傳統醫學科誠徵契約主治醫師</a></td>
 <td data-th="報名截止日">2023/1/10</td>
 </tr>,
 <tr>
 <td data-th="更新日期">2023/1/4</td>
 <td data-th="徵才類別">其他人員</td>
 <td class="aboutNews" data-th="徵才項目"><a href="/Module/RecruitmentContent?RecruitmentID=ab1bae10-ff31-44bd-91a0-83f4e89eac69&amp;WebMenuID=9d005e46-411b-46fc-b438-e0bd561eba78">耳鼻喉頭頸部研究助理徵才公告</a></td>
 <td data-th="報名截止日">2023/1/11</td>
 </tr>,
 <tr>
 <td data-th="更新日期">2023/1/4</td>
 <td data-th="徵才類別">技術人員</td>
 <td class="aboutNews" data-th="徵才項目"><a href="/Module/RecruitmentContent?RecruitmentID=96e8b638-6b7a-4bd2-8a1e-aaf7aa96

In [4]:
for tt in soup.find('table').find_all('tr'):
    for ttt in tt.findAll('td'):
        if ttt.get('data-th') == '更新日期':
            print('data',ttt.text)
        elif ttt.get('data-th') == '徵才類別':
            print('召聘職稱',ttt.text)
        elif ttt.get('data-th') == '報名截止日':
            print('期限' , ttt.text)
        elif ttt.get('data-th') == '徵才項目':
            job_detail_link = ttt.find('a')
            print('詳細連結', url_base+job_detail_link.get('href'))
        else:
            print(ttt)


data 2023/1/5
召聘職稱 醫師
詳細連結 https://www.vghtc.gov.tw/Module/RecruitmentContent?RecruitmentID=6af62829-4ce3-4f1f-bc9a-4a9d8317df80&WebMenuID=9d005e46-411b-46fc-b438-e0bd561eba78
期限 2023/1/10
data 2023/1/4
召聘職稱 其他人員
詳細連結 https://www.vghtc.gov.tw/Module/RecruitmentContent?RecruitmentID=ab1bae10-ff31-44bd-91a0-83f4e89eac69&WebMenuID=9d005e46-411b-46fc-b438-e0bd561eba78
期限 2023/1/11
data 2023/1/4
召聘職稱 技術人員
詳細連結 https://www.vghtc.gov.tw/Module/RecruitmentContent?RecruitmentID=96e8b638-6b7a-4bd2-8a1e-aaf7aa964cba&WebMenuID=9d005e46-411b-46fc-b438-e0bd561eba78
期限 2023/1/10
data 2023/1/4
召聘職稱 醫師
詳細連結 https://www.vghtc.gov.tw/Module/RecruitmentContent?RecruitmentID=ea2fec17-d919-4ce8-a706-1ea46102d838&WebMenuID=9d005e46-411b-46fc-b438-e0bd561eba78
期限 2023/1/31
data 2023/1/4
召聘職稱 住院醫師
詳細連結 https://www.vghtc.gov.tw/Module/RecruitmentContent?RecruitmentID=53042725-a656-4729-8547-97ae806dc9b5&WebMenuID=9d005e46-411b-46fc-b438-e0bd561eba78
期限 2023/2/1
data 2022/12/26
召聘職稱 其他人員
詳細連結 https://www.vghtc.g

In [5]:
work_table=[]
for tt in soup.find('table').find_all('tr'):
    _one_job_data ={}
    for ttt in tt.findAll('td'):
        
        if ttt.get('data-th') == '更新日期':
            #print('data',ttt.text)
            pass
        elif ttt.get('data-th') == '報名截止日':
            #print('期限' , ttt.text)
            _one_job_data['deadline'] = ttt.text
        elif ttt.get('data-th') == '徵才項目':
            clean_text = ttt.text.replace('【徵才公告】','').replace("【徵的就是你/妳】","")
            _one_job_data['jobtype'] = clean_text
            try:
                _one_job_data['job_type'] = re.search(r"[放,復,營,資,護,家,胸,皮,泌,眼,內,外,藥,麻,臨,急,病,兒,環,健]\w+[科,部,室,心,課]",clean_text).group(0)
            except:
                print(ttt.text,'error find job_type')
            job_detail_link = url_base+ttt.find('a').get('href')
            #print('詳細連結', job_detail_link)
            _one_job_data['job_detail_link'] = job_detail_link
            g=requests.get(job_detail_link)
            _soup=BeautifulSoup(g.content, 'html.parser')
            try:
                job_file_link = url_base+_soup.find('a',_class="btn-recruit-add").get('href')
            except:
                job_file_link = None
            _one_job_data['job_file_link'] = job_file_link
    print(_one_job_data)
    if _one_job_data:
        work_table.append(_one_job_data)
    print(pd.DataFrame(work_table))

{}
Empty DataFrame
Columns: []
Index: []
傳統醫學科誠徵契約主治醫師 error find job_type
{'jobtype': '傳統醫學科誠徵契約主治醫師', 'job_detail_link': 'https://www.vghtc.gov.tw/Module/RecruitmentContent?RecruitmentID=6af62829-4ce3-4f1f-bc9a-4a9d8317df80&WebMenuID=9d005e46-411b-46fc-b438-e0bd561eba78', 'job_file_link': None, 'deadline': '2023/1/10'}
         jobtype                                    job_detail_link  \
0  傳統醫學科誠徵契約主治醫師  https://www.vghtc.gov.tw/Module/RecruitmentCon...   

  job_file_link   deadline  
0          None  2023/1/10  
耳鼻喉頭頸部研究助理徵才公告 error find job_type
{'jobtype': '耳鼻喉頭頸部研究助理徵才公告', 'job_detail_link': 'https://www.vghtc.gov.tw/Module/RecruitmentContent?RecruitmentID=ab1bae10-ff31-44bd-91a0-83f4e89eac69&WebMenuID=9d005e46-411b-46fc-b438-e0bd561eba78', 'job_file_link': None, 'deadline': '2023/1/11'}
          jobtype                                    job_detail_link  \
0   傳統醫學科誠徵契約主治醫師  https://www.vghtc.gov.tw/Module/RecruitmentCon...   
1  耳鼻喉頭頸部研究助理徵才公告  https://www.vghtc.gov.tw/Modu

In [6]:
pd.DataFrame(work_table)

,jobtype,job_detail_link,job_file_link,deadline,job_type
0,傳統醫學科誠徵契約主治醫師,https://www.vghtc.gov.tw/Module/RecruitmentCon...,None,2023/1/10,NaN
1,耳鼻喉頭頸部研究助理徵才公告,https://www.vghtc.gov.tw/Module/RecruitmentCon...,None,2023/1/11,NaN
2,【病理檢驗部】微生物科誠徵定期契約醫事檢驗師(職務代理),https://www.vghtc.gov.tw/Module/RecruitmentCon...,None,2023/1/10,病理檢驗部
3,【病理檢驗部】誠徵解剖病理科契約主治醫師,https://www.vghtc.gov.tw/Module/RecruitmentCon...,None,2023/1/31,病理檢驗部
4,兒童醫學中心招訓112年住院醫師甄選(延長公告),https://www.vghtc.gov.tw/Module/RecruitmentCon...,None,2023/2/1,兒童醫學中心
5,契約醫事技術師(物理治療師3名),https://www.vghtc.gov.tw/Module/RecruitmentCon...,None,2023/1/16,NaN
6,放射線部招訓110、111年度住院醫師甄選公告,https://www.vghtc.gov.tw/Module/RecruitmentCon...,None,2023/1/9,放射線部
7,臺中榮民總醫院急診部急診醫學科招訓110年度第二年(雙專長)住院醫師甄選公告,https://www.vghtc.gov.tw/Module/RecruitmentCon...,None,2023/2/15,急診部急診醫學科
8,臺中榮民總醫院急診部急診醫學科招訓111年度第一年住院醫師甄選公告,https://www.vghtc.gov.tw/Module/RecruitmentCon...,None,2023/2/15,急診部急診醫學科
9,急診部急診醫學科招訓112年度第一年住院醫師-延長公告,https://www.vghtc.gov.tw/Module/RecruitmentCon...,None,2023/2/15,急診部急診醫學科


# Get page link
In this website the work table has some page, so the function should find page link for next step.

In [9]:
pages_link = []

pages_link.append(url)

for pp in soup.find('div', class_="pager").find_all('a'):
    if pp.get('title') == 'Next page':
        pass
    else:
        pages_link.append(url_base+pp.get('href'))
pages_link

['https://www.vghtc.gov.tw/Module/Recruitment?WebMenuID=9d005e46-411b-46fc-b438-e0bd561eba78',
 'https://www.vghtc.gov.tw/Module/Recruitment?WebMenuID=9d005e46-411b-46fc-b438-e0bd561eba78&page=2']

# Get admit table

The get_work_page function is used for getting each work page


In [5]:
url=url_base+url_admit

g=requests.get(url)

admit_page_base=BeautifulSoup(g.content, 'html.parser')


admit_tables = admit_page_base.find_all('tr')

In [6]:
admit_tables

[<tr>
 <th><div style="text-align:left;">徵才項目</div></th>
 <th><div style="text-align:left;">錄取名單</div></th>
 <th><div style="text-align:left;">公告迄日</div></th>
 </tr>,
 <tr>
 <td class="aboutNews" data-th="徵才項目"><a href="/Module/RecruitmentContent?RecruitmentID=fa229ed5-b58b-4734-ac59-02fb166c8240&amp;WebMenuID=9d005e46-411b-46fc-b438-e0bd561eba78">【錄取公告】皮膚科招訓112年度第一年住院醫師</a></td>
 <td class="aboutNews" data-th="錄取名單"><a href="/UploadFiles/Files/RecruitAttach/臺中榮總皮膚科112年度第一年住院醫師錄取名單.jpg" target="_blank">正取1名/備取2名</a></td>
 <td data-th="公告迄日">2023/3/21</td>
 </tr>,
 <tr>
 <td class="aboutNews" data-th="徵才項目"><a href="/Module/RecruitmentContent?RecruitmentID=c5c35d4e-bd26-41bc-b3af-88b12f71525d&amp;WebMenuID=9d005e46-411b-46fc-b438-e0bd561eba78">台中榮總家庭醫學部112年第一年住院醫師</a></td>
 <td class="aboutNews" data-th="錄取名單"><a href="/UploadFiles/Files/RecruitAttach/臺中榮總家庭醫學部112年第一年住院醫師錄取名單.pdf" target="_blank">4名</a></td>
 <td data-th="公告迄日">2023/3/7</td>
 </tr>,
 <tr>
 <td class="aboutNews" data-th=

In [21]:
admit_table = []
for tt in admit_tables:
    _admit_data = {}
    for ttt in tt.findAll('td'):
        if ttt.get('data-th') == "徵才項目":
            _admit_data['召聘職稱'] = ttt.text.replace("【錄取公告】","")
        elif ttt.get('data-th') == '公告迄日':
            _admit_data['期限'] = ttt.text
        elif ttt.get('data-th') == '錄取名單':
            job_detail_link = ttt.find('a')
            _admit_data['連結'] = url_base+job_detail_link.get('href')
        else:
            print("Error",ttt, "other message")
    if _admit_data != {}:
        admit_table.append(_admit_data)

In [22]:
admit_table

[{'召聘職稱': '皮膚科招訓112年度第一年住院醫師',
  '連結': 'https://www.vghtc.gov.tw/UploadFiles/Files/RecruitAttach/臺中榮總皮膚科112年度第一年住院醫師錄取名單.jpg',
  '期限': '2023/3/21'},
 {'召聘職稱': '台中榮總家庭醫學部112年第一年住院醫師',
  '連結': 'https://www.vghtc.gov.tw/UploadFiles/Files/RecruitAttach/臺中榮總家庭醫學部112年第一年住院醫師錄取名單.pdf',
  '期限': '2023/3/7'},
 {'召聘職稱': '藥學部誠徵契約(含定期)藥師',
  '連結': 'https://www.vghtc.gov.tw/UploadFiles/Files/RecruitAttach/',
  '期限': '2023/2/1'},
 {'召聘職稱': '胸腔內科徵：師三級主治醫師1名',
  '連結': 'https://www.vghtc.gov.tw/UploadFiles/Files/RecruitAttach/',
  '期限': '2023/1/3'},
 {'召聘職稱': '【徵才公告】醫院整合醫學暨醫療銜接照護試辦計畫研究助理1名',
  '連結': 'https://www.vghtc.gov.tw/UploadFiles/Files/RecruitAttach/錄取醫院整合醫學暨醫療銜接照護試辦計畫專任助理一名.pdf',
  '期限': '2022/12/28'},
 {'召聘職稱': '藥學部誠徵契約(含定期)藥師',
  '連結': 'https://www.vghtc.gov.tw/UploadFiles/Files/RecruitAttach/',
  '期限': '2022/11/30'},
 {'召聘職稱': '藥學部誠徵半日契約藥師1名',
  '連結': 'https://www.vghtc.gov.tw/UploadFiles/Files/RecruitAttach/',
  '期限': '2022/10/26'},
 {'召聘職稱': '藥學部誠徵定期契約藥師2名',
  '連結': 'https://www.vghtc.gov.t

In [23]:
pd.DataFrame(admit_table, columns=['召聘職稱','期限' ,'連結'])

,召聘職稱,期限,連結
0,皮膚科招訓112年度第一年住院醫師,2023/3/21,https://www.vghtc.gov.tw/UploadFiles/Files/Rec...
1,台中榮總家庭醫學部112年第一年住院醫師,2023/3/7,https://www.vghtc.gov.tw/UploadFiles/Files/Rec...
2,藥學部誠徵契約(含定期)藥師,2023/2/1,https://www.vghtc.gov.tw/UploadFiles/Files/Rec...
3,胸腔內科徵：師三級主治醫師1名,2023/1/3,https://www.vghtc.gov.tw/UploadFiles/Files/Rec...
4,【徵才公告】醫院整合醫學暨醫療銜接照護試辦計畫研究助理1名,2022/12/28,https://www.vghtc.gov.tw/UploadFiles/Files/Rec...
5,藥學部誠徵契約(含定期)藥師,2022/11/30,https://www.vghtc.gov.tw/UploadFiles/Files/Rec...
6,藥學部誠徵半日契約藥師1名,2022/10/26,https://www.vghtc.gov.tw/UploadFiles/Files/Rec...
7,藥學部誠徵定期契約藥師2名,2022/10/26,https://www.vghtc.gov.tw/UploadFiles/Files/Rec...
8,【徵才公告】胸腔內科徵：契約醫事技術師1名,2022/9/9,https://www.vghtc.gov.tw/UploadFiles/Files/Rec...
9,\t【徵才公告】品質管理中心誠徵契約醫務管理專員一名,2022/8/5,https://www.vghtc.gov.tw/UploadFiles/Files/Rec...
